# An Introduction to JAX

----

#### John Stachurski

#### Prepared for the CBC Workshop (May 2024)

----


This lecture provides a short introduction to [Google JAX](https://github.com/google/jax).

What GPUs do we have access to?

In [1]:
!nvidia-smi

Fri May 10 13:25:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        Off |   00000000:3E:00.0 Off |                  N/A |
| 30%   29C    P8             29W /  320W |    7826MiB /  10240MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## JAX as a NumPy Replacement


One way to use JAX is as a plug-in NumPy replacement. Let's look at the
similarities and differences.

### Similarities


The following import is standard, replacing `import numpy as np`:

In [2]:
import jax
import jax.numpy as jnp

Now we can use `jnp` in place of `np` for the usual array operations:

In [3]:
a = jnp.array((1.0, 3.2, -1.5))

In [4]:
print(a)

[ 1.   3.2 -1.5]


In [5]:
print(jnp.sum(a))

2.6999998


In [6]:
print(jnp.mean(a))

0.9


In [7]:
print(jnp.dot(a, a))

13.490001


In [8]:
print(a @ a)  # Equivalent

13.490001


However, the array object `a` is not a NumPy array:

In [9]:
a

Array([ 1. ,  3.2, -1.5], dtype=float32)

In [10]:
type(a)

jaxlib.xla_extension.ArrayImpl

Even scalar-valued maps on arrays return JAX arrays.

In [11]:
jnp.sum(a)

Array(2.6999998, dtype=float32)

JAX arrays are also called "device arrays," where term "device" refers to a
hardware accelerator (GPU or TPU).

(In the terminology of GPUs, the "host" is the machine that launches GPU operations, while the "device" is the GPU itself.)



Operations on higher dimensional arrays are also similar to NumPy:

In [12]:
A = jnp.ones((2, 2))
B = jnp.identity(2)
A @ B

Array([[1., 1.],
       [1., 1.]], dtype=float32)

In [13]:
jnp.linalg.inv(B)   # Inverse of identity is identity

RuntimeError: jaxlib/gpu/solver_kernels.cc:45: operation gpusolverDnCreate(&handle) failed: cuSolver internal error

In [ ]:
result = linalg.eigh(B)  # Computes eigenvalues and eigenvectors
result.eigenvalues

In [ ]:
result.eigenvectors

### Differences

Let's now look at the differences between JAX and NumPy

#### 32 bit floats

One difference between NumPy and JAX is that JAX currently uses 32 bit floats by default.  

This is standard for GPU computing and can lead to significant speed gains with small loss of precision.

However, for some calculations precision matters.  In these cases 64 bit floats can be enforced via the command

In [ ]:
jax.config.update("jax_enable_x64", True)

Let's check this works:

In [ ]:
jnp.ones(3)

#### Mutability

As a NumPy replacement, a more significant difference is that arrays are treated as **immutable**.  

For example, with NumPy we can write

In [ ]:
import numpy as np
a = np.linspace(0, 1, 3)
a

and then mutate the data in memory:

In [ ]:
a[0] = 1
a

In JAX this fails:

In [ ]:
a = jnp.linspace(0, 1, 3)
a

In [ ]:
a[0] = 1   # uncommenting produces a TypeError

The designers of JAX chose to make arrays immutable because JAX uses a
functional programming style.  More on this below.  

#### Sneaky mutation

Note that, while mutation is discouraged, it is in fact possible with `at`, as in

In [ ]:
a = jnp.linspace(0, 1, 3)
id(a)

In [ ]:
a

In [ ]:
a.at[0].set(1)

We can check that the array is mutated by verifying its identity is unchanged:

In [ ]:
id(a)

In general it's better to avoid mutating arrays --- more discussion below.

## Random Numbers

Random numbers are also a bit different in JAX, relative to NumPy.  


### Controlling the state

Typically, in JAX, the state of the random number generator needs to be controlled explicitly.

(This is also related to JAX's functional programming paradigm, discussed below.  JAX does not typically work with objects that maintain state, such as the state of a random number generator.)

In [ ]:
import jax.random as random

First we produce a key, which seeds the random number generator.

In [ ]:
key = random.PRNGKey(1)

In [ ]:
type(key)

In [ ]:
print(key)

Now we can use the key to generate some random numbers:

In [ ]:
x = random.normal(key, (3, 3))
x

If we use the same key again, we initialize at the same seed, so the random numbers are the same:

In [ ]:
random.normal(key, (3, 3))

### Generating fresh draws

To produce a (quasi-) independent draw, we can use `split`

In [ ]:
new_keys = random.split(key, 5)   # Generate 5 new keys

In [ ]:
len(new_keys)

In [ ]:
for key in new_keys:
    print(random.normal(key, (3, )))

Another function we can use to update the key is `fold_in`.

In [ ]:
seed = 1234  # seed to generate new key from old
key = jax.random.fold_in(key, seed)

random.normal(key, (3, 1))

This is often used in loops -- here's an example that produces `k` (quasi-) independent random `n x n` matrices using this procedure and prints their determinants.

In [ ]:
def gen_random_matrices(seed=1234, n=10, k=5):
    key = random.PRNGKey(seed)
    for i in range(k):
        key = random.fold_in(key, i)
        d = jnp.linalg.det(random.uniform(key, (n, n)))
        print(f"Determinant = {d:.4}")

gen_random_matrices()

## JIT compilation

The JAX just-in-time (JIT) compiler generates efficient, parallelized machine code optimized for either the CPU or the GPU/TPU, depending on whether one of these accelerators is detected.

### A first example

To see the JIT compiler in action, consider the following function.

In [ ]:
def f(x):
    a = 3*x + jnp.sin(x) + jnp.cos(x**2) - jnp.cos(2*x) - x**2 * 0.4 * x**1.5
    return jnp.sum(a)

Let's build an array to call the function on.

In [ ]:
n = 50_000_000
x = jnp.ones(n)

How long does the function take to execute?

In [ ]:
%time f(x).block_until_ready()

(In order to measure actual speed, we use `block_until_ready()` method 
to hold the interpreter until the results of the computation are returned from
the device. This is necessary because JAX uses asynchronous dispatch, which
allows the Python interpreter to run ahead of GPU computations.)

The code doesn't run as fast as we might hope, given that it's running on a GPU.

But if we run it a second time it becomes much faster:

In [ ]:
%time f(x).block_until_ready()

In [ ]:
%timeit f(x).block_until_ready()

This is because the built in functions like `jnp.cos` are JIT compiled and the
first run includes compile time.

### When does JAX recompile?

You might remember that Numba recompiles if we change the types of variables in a function call.

JAX recompiles more often --- in particular, it recompiles every time we change array sizes.

For example, let's try

In [ ]:
m = n + 1
y = jnp.ones(m)

In [ ]:
%time f(y).block_until_ready()

Notice that the execution time increases, because now new versions of 
the built-ins like `jnp.cos` are being compiled, specialized to the new array
size.

If we run again, the code is dispatched to the correct compiled version and we
get faster execution.

In [ ]:
%time f(y).block_until_ready()

Why does JAX generate fresh machine code every time we change the array size???

The compiled versions for the previous array size are still available in memory
too, and the following call is dispatched to the correct compiled code.

In [ ]:
%time f(x).block_until_ready()

### Compiling user-built functions

We can instruct JAX to compile entire functions that we build.

For example, consider

In [ ]:
def g(x):
    y = jnp.zeros_like(x)
    for i in range(10):
        y += x**i
    return y

In [ ]:
n = 1_000_000
x = jnp.ones(n)

Let's time it.

In [ ]:
%time g(x).block_until_ready()

In [ ]:
%time g(x).block_until_ready()

In [ ]:
g_jit = jax.jit(g)   # target for JIT compilation

Let's run once to compile it:

In [ ]:
g_jit(x)

And now let's time it.

In [ ]:
%time g_jit(x).block_until_ready()

Note the speed gain.

This is because 

1. the loop is compiled and
2. the array operations are fused and no intermediate arrays are created.


Incidentally, a more common syntax when targetting a function for the JIT
compiler is

In [ ]:
@jax.jit
def g_jit_2(x):
    y = jnp.zeros_like(x)
    for i in range(10):
        y += x**i
    return y

In [ ]:
%time g_jit_2(x).block_until_ready()

In [ ]:
%time g_jit_2(x).block_until_ready()

#### Static arguments

Because the compiler specializes on array sizes, it needs to recompile code when array sizes change.

As a result, any argument that determines sizes of arrays should be flagged by `static_argnums` -- a signal that JAX can treat that variable as a compile-time constant (and recompile when it changes).

Here's a example.

In [ ]:
def f(n, seed=1234):
    key = jax.random.PRNGKey(seed)
    x = jax.random.normal(key, (n, ))
    return x.std()

In [ ]:
f(5)

In [ ]:
f_jitted = jax.jit(f)

In [ ]:
f_jitted(5)

Let's fix this:

In [ ]:
f_jitted = jax.jit(f, static_argnums=(0, ))   # First argument is static
f_jitted(5)

## Functional Programming

From JAX's documentation:

*When walking about the countryside of Italy, the people will not hesitate to tell you that JAX has “una anima di pura programmazione funzionale”.*


In other words, JAX assumes a functional programming style.

The major implication is that JAX functions should be pure.
    
A pure function will always return the same result if invoked with the same inputs.

In particular, a pure function has

* no dependence on global variables and
* no side effects

### Examples: Python/NumPy/Numba style code is not pure

#### Example 1

Here's an example to show that NumPy functions are not pure:

In [ ]:
np.random.randn()

In [ ]:
np.random.randn()

This function returns the different results when called on the same inputs!

The issue is that the function maintains state between function calls --- the state of the random number generator.

#### Example 2

Here's a function that's not pure because it depends on a global

In [ ]:
a = 10
def f(x): return a * x

f(1)

In [ ]:
a = 20
f(1)

(Notice that the output of the function cannot be fully predicted from the inputs.)

#### Example 3

Here's a function that fails to be pure because it modifies external state.

In [ ]:
def double_input(x):   # Not pure -- side effects
    x[:] = 2 * x
    return None

x = np.ones(5)
x

In [ ]:
double_input(x)
x

Here's a pure version:

In [ ]:
def double_input(x):
    y = 2 * x
    return y

#### Example 4

The following function is also not pure, since it modifies a global variable (similar to the last example).

In [ ]:
a = 1
def f():
    global a
    a += 1
    return None

In [ ]:
a

In [ ]:
f()

In [ ]:
a

### Compiling impure functions

JAX does not insist on pure functions.

For example, JAX will not usually throw errors when compiling impure functions 

However, execution becomes unpredictable!

Here's an illustration of this fact, using global variables:

In [ ]:
a = 1  # global

@jax.jit
def f(x):
    return a + x

In [ ]:
x = jnp.ones(2)

In [ ]:
x

In [ ]:
f(x)

In the code above, the global value `a=1` is fused into the jitted function.

Even if we change `a`, the output of `f` will not be affected --- as long as the same compiled version is called.

In [ ]:
a = 42

In [ ]:
f(x)

Notice that the change in the value of `a` takes effect in the code below:

In [ ]:
x = jnp.ones(3)

In [ ]:
f(x)

Can you explain why?

#### Moral

Moral of the story: write pure functions when using JAX!

## Gradients

JAX can use automatic differentiation to compute gradients.

This can be extremely useful for optimization and solving nonlinear systems.

We will see significant applications later in this lecture series.

For now, here's a very simple illustration involving the function

In [ ]:
def f(x):
    return (x**2) / 2

Let's take the derivative:

In [ ]:
f_prime = jax.grad(f)

In [ ]:
f_prime(10.0)

Let's plot the function and derivative, noting that $f'(x) = x$.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
x_grid = jnp.linspace(-4, 4, 200)
ax.plot(x_grid, f(x_grid), label="$f$")
ax.plot(x_grid, [f_prime(x) for x in x_grid], label="$f'$")
ax.legend(loc='upper center')
plt.show()

## Writing vectorized code

Writing fast JAX code requires shifting repetitive tasks from loops to array processing operations, so that the JAX compiler can easily understand the whole operation and generate more efficient machine code.

This procedure is called **vectorization** or **array programming**, and will be familiar to anyone who has used NumPy or MATLAB.

In some ways, vectorization is the same in JAX as it is in NumPy.

But there are also differences, which we highlight here.

As a running example, consider the function

$$
    f(x,y) = \frac{\cos(x^2 + y^2)}{1 + x^2 + y^2}
$$

Suppose that we want to evaluate this function on a square grid of $x$ and $y$ points.


### A slow version with loops

To clarify, here is the slow `for` loop version, which we run in a setting where `len(x) = len(y)` is very small.

In [ ]:
@jax.jit
def f(x, y):
    return jnp.cos(x**2 + y**2) / (1 + x**2 + y**2)

n = 80
x = jnp.linspace(-2, 2, n)
y = x

z_loops = np.empty((n, n))

In [ ]:
%%time
for i in range(n):
    for j in range(n):
        z_loops[i, j] = f(x[i], y[j])

Even for this very small grid, the run time is extremely slow.

(Notice that we used a NumPy array for `z_loops` because we wanted to write to it.)

OK, so how can we do the same operation in vectorized form?

If you are new to vectorization, you might guess that we can simply write

In [ ]:
z_bad = f(x, y)

But this gives us the wrong result because JAX doesn't understand the nested for loop.

In [ ]:
z_bad.shape

Here is what we actually wanted:

In [ ]:
z_loops.shape

### Vectorization attempt 1

To get the right shape and the correct nested for loop calculation, we can use a `meshgrid` operation that originated in MATLAB and was replicated in NumPy and then JAX:

In [ ]:
x_mesh, y_mesh = jnp.meshgrid(x, y)

Now we get what we want and the execution time is fast.

In [ ]:
z_mesh = f(x_mesh, y_mesh) 

Let's confirm that we got the right answer.

In [ ]:
jnp.allclose(z_mesh, z_loops)

Now we can set up a serious grid and run the same calculation (on the larger grid) in a short amount of time.

In [ ]:
n = 6000
x = jnp.linspace(-2, 2, n)
y = x
x_mesh, y_mesh = jnp.meshgrid(x, y)

In [ ]:
%%time
z_mesh = f(x_mesh, y_mesh) 

In [ ]:
%%time
z_mesh = f(x_mesh, y_mesh) 

But there is one problem here: the mesh grids use a lot of memory.

In [ ]:
(x_mesh.nbytes + y_mesh.nbytes) / 1_000_000  # MB of memory

By comparison, the flat array `x` is just

In [ ]:
x.nbytes / 1_000_000   # and y is just a pointer to x

This extra memory usage can be a big problem in actual research calculations.

In [ ]:
del x_mesh  # Free memory
del y_mesh  # Free memory

### Vectorization attempt 2

We can achieve a similar effect through NumPy style broadcasting rules.

In [ ]:
x_reshaped = jnp.reshape(x, (n, 1))   # Give x another dimension (column)
y_reshaped = jnp.reshape(y, (1, n))   # Give y another dimension (row)

When we evaluate $f$ on these reshaped arrays, we replicate the nested for loops in the original version.

In [ ]:
%time z_reshaped = f(x_reshaped, y_reshaped)

In [ ]:
%time z_reshaped = f(x_reshaped, y_reshaped)

Let's check that we got the same result

In [ ]:
jnp.allclose(z_reshaped, z_mesh)

The memory usage for the inputs is much more moderate.

In [ ]:
(x_reshaped.nbytes + y_reshaped.nbytes) / 1_000_000

### Vectorization attempt 3


There's another approach to vectorization we can pursue, using [jax.vmap](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html)

It runs out that, when we are working with complex functions and operations, this `vmap` approach can be the easiest to implement.

It's also very memory parsimonious.

The first step is to vectorize the function `f` in `y`.

In [ ]:
f_vec_y = jax.vmap(f, in_axes=(None, 0))  

In the line above, `(None, 0)` indicates that we are vectorizing in the second argument, which is `y`.

Next, we vectorize in the first argument, which is `x`.

In [ ]:
f_vec = jax.vmap(f_vec_y, in_axes=(0, None))

Finally, we JIT-compile the result:

In [ ]:
f_vec = jax.jit(f_vec)

With this construction, we can now call the function $f$ on flat (low memory) arrays.

In [ ]:
%%time
z_vmap = f_vec(x, y)

In [ ]:
%%time
z_vmap = f_vec(x, y)

Let's check we produce the correct answer:

In [ ]:
jnp.allclose(z_vmap, z_mesh)

Let's finish by cleaning up.

In [ ]:
del z_mesh
del z_vmap
del z_reshaped

### Exercises

Repeat the exercise of computing the approximation to $\pi$ by simulation:

1. draw $n$ observations of a bivariate uniform on the unit square
2. count the fraction that fall in the unit circle (radius 0.5) centered on (0.5, 0.5)
3. multiply the result by 4

Use JAX

In [ ]:
for i in range(12):
    print("Solution below 🐠")

In [ ]:
def approx_pi(n, key):
    u = jax.random.uniform(key, (2, n))
    distances = jnp.sqrt((u[0, :] - 0.5)**2 + (u[1, :] - 0.5)**2)
    fraction_in_circle = jnp.mean(distances < 0.5)
    return fraction_in_circle * 4  # dividing by radius**2

n = 1_000_000 # sample size for Monte Carlo simulation
key = jax.random.PRNGKey(1234)

In [ ]:
%time approx_pi(n, key)

In [ ]:
%time approx_pi(n, key)

In [ ]:
approx_pi_jitted = jax.jit(approx_pi, static_argnums=(0,))

In [ ]:
%time approx_pi_jitted(n, key)

In [ ]:
%time approx_pi_jitted(n, key)

**Exercise**

In a previous notebook we used Monte Carlo to price a European call option and
constructed a solution using Numba.

The code looked like this:

In [ ]:
import numba
from numpy.random import randn
M = 10_000_000

n, β, K = 20, 0.99, 100
μ, ρ, ν, S0, h0 = 0.0001, 0.1, 0.001, 10, 0

@numba.jit(parallel=True)
def compute_call_price_parallel(β=β,
                                μ=μ,
                                S0=S0,
                                h0=h0,
                                K=K,
                                n=n,
                                ρ=ρ,
                                ν=ν,
                                M=M):
    current_sum = 0.0
    # For each sample path
    for m in numba.prange(M):
        s = np.log(S0)
        h = h0
        # Simulate forward in time
        for t in range(n):
            s = s + μ + np.exp(h) * randn()
            h = ρ * h + ν * randn()
        # And add the value max{S_n - K, 0} to current_sum
        current_sum += np.maximum(np.exp(s) - K, 0)
        
    return β**n * current_sum / M

Let's run it once to compile it:

In [ ]:
compute_call_price_parallel()

And now let's time it:

In [ ]:
%%time 
compute_call_price_parallel()

Try writing a version of this operation for JAX, using all the same
parameters.

If you are running your code on a GPU, you should be able to achieve
significantly faster execution.

In [ ]:
for i in range(12):
    print("Solution below 🐠")

**Solution**

Here is one solution:

In [ ]:
@jax.jit
def compute_call_price_jax(β=β,
                           μ=μ,
                           S0=S0,
                           h0=h0,
                           K=K,
                           n=n,
                           ρ=ρ,
                           ν=ν,
                           M=M,
                           key=jax.random.PRNGKey(1)):

    s = jnp.full(M, np.log(S0))
    h = jnp.full(M, h0)
    for t in range(n):
        key, subkey = jax.random.split(key)
        Z = jax.random.normal(subkey, (2, M))
        s = s + μ + jnp.exp(h) * Z[0, :]
        h = ρ * h + ν * Z[1, :]
    expectation = jnp.mean(jnp.maximum(jnp.exp(s) - K, 0))
        
    return β**n * expectation

Let's run it once to compile it:

In [ ]:
compute_call_price_jax()

And now let's time it:

In [ ]:
%%time 
compute_call_price_jax().block_until_ready()